In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
import numpy as np

In [ ]:
# usar aca el path a la carpeta con datos descomprimidos
data_dir = Path("D:/transit-data/data-SNBE")

In [ ]:
# esta celda tarda MUCHO TIEMPO, una mejor opción es usar la instancia de BigQuery 
combined_dataset = data_dir / "combined.pkl"
if not combined_dataset.exists():
    weekdays = ["Do","Lu", "Ma", "Mi", "Ju", "Vi", "Sa"]
    D = pd.concat([pd.read_csv(i, sep=";") for i in data_dir.glob("*.csv")])
    D["fechahoraevento"] = pd.to_datetime(D["fechahoraevento"])
    D["ymd"] = D["fechahoraevento"].apply(lambda x: f"{x.date()} ({weekdays[x.weekday()]})")
    D["year"] = D["fechahoraevento"].apply(lambda x: x.year)
    D["month"] = D["fechahoraevento"].apply(lambda x: x.month)
    D["day"] = D["fechahoraevento"].apply(lambda x: x.day)
    D["hour"] = D["fechahoraevento"].apply(lambda x: x.hour)
    D.to_pickle(combined_dataset)
else:
    D = pd.read_pickle(combined_dataset)

In [ ]:
# usuarios por día
usuario_por_dia = D[["ymd", "serialtarjeta", "idrutaestacion"]].groupby(["ymd", "idrutaestacion"])["serialtarjeta"].nunique().reset_index()
usuario_por_dia


In [ ]:
idsam_idrutaestacion = D[["idsam", "idrutaestacion"]].drop_duplicates()

In [ ]:
tarj_dia = D[["serialtarjeta", "ymd"]].copy()

In [ ]:
tarj_dia["val"] = 1
usos_tarj_dia = tarj_dia.groupby(["serialtarjeta", "ymd"]).count()

In [ ]:
count_usos_tarj_dia = usos_tarj_dia.reset_index().sort_values("val", ascending=False)

In [ ]:
count_usos_tarj_dia_top50 = count_usos_tarj_dia[:50]

In [ ]:
data = {k:[] for k in ["serialtarjeta", "ymd", "montoevento", "tipoevento", "idrutaestacion"]}
for i, r in count_usos_tarj_dia_top50.iterrows():
    cond = (D["serialtarjeta"] == r["serialtarjeta"]) & (D["ymd"] == r["ymd"])
    d = D[cond]
    for _, rr in d.iterrows():
        data["serialtarjeta"].append(rr["serialtarjeta"])
        data["ymd"].append(rr["ymd"])
        data["montoevento"].append(rr["montoevento"])
        data["tipoevento"].append(rr["tipoevento"])
        data["idrutaestacion"].append(rr["idrutaestacion"])
top_spenders = pd.DataFrame(data)

In [ ]:
tarjeta = "" #rellenar con la tarjeta a analizar
dia = "2022-04-25 (Do)"
cond = (D["serialtarjeta"] == tarjeta) & (D["ymd"] == dia)


In [ ]:
f, ax = plt.subplots(figsize=(10, 5))
sns.histplot(D[cond], x="hour", bins=range(24))
ax.set_ylabel("cantidad de eventos")
ax.set_xlabel("hora")
ax.set_title(f"Cantidad de eventos para tarjeta <ANONIMIZADO> el {dia}\n# de eventos: {D[cond].shape[0]} monto: {D[cond]['montoevento'].sum()}")
plt.show()
plt.close("all")


In [ ]:
top_spenders.groupby(["serialtarjeta", "ymd"]).agg({"montoevento":"sum", "tipoevento":"count", "idrutaestacion":"nunique"})

In [ ]:
D[D["serialtarjeta"] == "18f6e00ec04d8e05438254c3b1e8502b"].to_csv("biggest_spender.csv")

In [ ]:
users_by_month.reset_index().to_csv("tarjeta_dia_mes.csv")

In [ ]:
cond = usuario_por_dia["idrutaestacion"].isin(['0091', '0016', '0155', '0090', '005b', '0143', '005c', '0145'])
d = usuario_por_dia[cond].pivot(index="ymd", columns="idrutaestacion", values="serialtarjeta").fillna(0)

cond = ~usuario_por_dia["idrutaestacion"].isin(['0000'])
d = usuario_por_dia[cond].pivot(index="ymd", columns="idrutaestacion", values="serialtarjeta").fillna(0)

cmap=plt.get_cmap("crest_r")
cmap.set_under("lightsalmon")
f, ax = plt.subplots(figsize=(30, 70))
sns.heatmap(d.astype(int).T, 
            annot=True,
            cmap=cmap,
            vmin=1,
            fmt=",",
            ax=ax)
ax.set_ylabel("día")
ax.set_xlabel("ID Ruta/Estación")
ax.set_title(f"Cantidad tarjetas activas por día e ID Ruta/Estación")
plt.savefig(f'usuarios-por-dia.pdf', bbox_inches='tight')
plt.show()
plt.close("all")

In [ ]:
dd = D[cond][["ymd", "hour", "idrutaestacion", "idsam"]].copy()
d = dd.groupby(["ymd", "hour", "idrutaestacion"])["idsam"].nunique().reset_index()

In [ ]:
rutaestacion = "005a"
cond = d["idrutaestacion"] == rutaestacion
d = d[cond].pivot(index="ymd", columns="hour", values="idsam").fillna(0)

In [ ]:
cmap=plt.get_cmap("crest_r")
cmap.set_under("lightsalmon")
f, ax = plt.subplots(figsize=(15, 10))
sns.heatmap(d.astype(int), 
            annot=True,
            cmap=cmap,
            vmin=1,
            fmt=",",
            ax=ax)
ax.set_ylabel("día")
ax.set_xlabel("hora")
ax.set_title(f"Cantidad de SAMs activos por día/hora para ID Ruta/Estación = {rutaestacion}")
plt.show()
plt.close("all")

In [ ]:
monto_by_x = D.groupby(["tipoevento", "tipotransporte", "producto", "year", "month"])["montoevento"].sum().reset_index()

In [ ]:
monto_by_x["ym"] = monto_by_x["year"].astype("str") + "/" + monto_by_x["month"].astype("str")

In [ ]:
g = sns.catplot(data=monto_by_x.sort_values(["year", "month"]),
                x="montoevento",
                y="ym",
                col="tipoevento",
                hue="tipotransporte",
                row="producto",
                kind="bar")
# iterate through axes
def to_int(x):
    if np.isnan(x):
        return 0
    return int(x)

for ax in g.axes.ravel():
    
    # add annotations
    for c in ax.containers:
        labels = [f' {to_int(v.get_width())/1000000:.1f}M' for v in c]
        ax.bar_label(c, labels=labels, label_type='edge', fontsize=6)
    #ax.margins(y=0.2)
    #ax.set_xscale("log")
#g.set_xticklabels(rotation=30)
plt.show()

In [ ]:
cond = (D["producto"] == "MO") & (D["tipoevento"] == 4)
tarj_counts = D[cond]["serialtarjeta"].value_counts()

In [ ]:
sns.kdeplot(tarj_counts)

In [ ]:
sns.kdeplot(tarj_counts[tarj_counts < 1000])